In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
!pip install tensorflow-gpu


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:

# Load the first dataset
df1 = pd.read_excel("Hackathon Dataset New.xlsx")

# Load the second dataset
df2 = pd.read_excel("Hackathon Dataset Extra100 cases.xlsx")

# Print column names of both datasets
print("Columns of df1:", df1.columns)
print("Columns of df2:", df2.columns)

Columns of df1: Index(['_id', 'Case_No', 'Date', 'Judge', 'pdf_url', 'Petitioner', 'Outcome',
       'Correct (Y/N)'],
      dtype='object')
Columns of df2: Index(['_id', 'Case Number', 'Petitioner Name', 'Judgment By', 'PDF Url',
       'Outcome', 'Correct (Y/N)'],
      dtype='object')


In [5]:
df2.rename(columns={'Case Number': 'Case_No', 'Judgment By': 'Judge','PDF Url':'pdf_url','Petitioner Name':'Petitioner'}, inplace=True)

In [6]:
# Drop the 'Date' column from df1
df1.drop(columns=['Date'], inplace=True)

In [7]:
df1.columns

Index(['_id', 'Case_No', 'Judge', 'pdf_url', 'Petitioner', 'Outcome',
       'Correct (Y/N)'],
      dtype='object')

In [8]:
df2.columns

Index(['_id', 'Case_No', 'Petitioner', 'Judge', 'pdf_url', 'Outcome',
       'Correct (Y/N)'],
      dtype='object')

In [9]:
desired_columns_order = ['_id', 'Case_No', 'Judge', 'pdf_url', 'Petitioner', 'Outcome',
       'Correct (Y/N)']

In [10]:
df2 = df2[desired_columns_order]


In [11]:
df2.columns

Index(['_id', 'Case_No', 'Judge', 'pdf_url', 'Petitioner', 'Outcome',
       'Correct (Y/N)'],
      dtype='object')

In [12]:


# Merge the datasets
df = pd.concat([df1, df2], ignore_index=True)

In [13]:
print(df.head())

                        _id                   Case_No  \
0  65deff06bf957c5aaed9145c      W.P.No.22072 of 2012   
1  65deff15bf957c5aaed9145e  W.P.(MD).No.2985 of 2015   
2  65deff20bf957c5aaed91460      W.P. No.8542 of 2015   
3  65deff2abf957c5aaed91462     CRL.RC.No.953 of 2011   
4  65deff34bf957c5aaed91464     CRL.RC.No.954 of 2011   

                                               Judge  \
0  The Hon'ble MR.SANJAY KISHAN KAUL THE CHIEF JU...   
1              THE HONOURABLE MR.JUSTICE B.RAJENDRAN   
2           THE HONOURABLE MR.JUSTICE S.VAIDYANATHAN   
3                  THE HON'BLE MR.JUSTICE C.T.SELVAM   
4                  THE HON'BLE MR.JUSTICE C.T.SELVAM   

                                             pdf_url  \
0  https://mhc.tn.gov.in/judis/index.php/casestat...   
1  https://mhc.tn.gov.in/judis/index.php/casestat...   
2  https://mhc.tn.gov.in/judis/index.php/casestat...   
3  https://mhc.tn.gov.in/judis/index.php/casestat...   
4  https://mhc.tn.gov.in/judis/index.php

In [14]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 2.3 MB/s eta 0:00:00


In [15]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00


In [16]:
!pip install sentencepiece

In [17]:
!pip install --upgrade certifi

In [18]:
!python -m certifi

/usr/local/lib/python3.10/dist-packages/certifi/cacert.pem


In [19]:
pip install --upgrade requests

In [20]:
import requests
from PyPDF2 import PdfReader
import pandas as pd

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_url):
    try:
        pdf_response = requests.get(pdf_url, verify=False)
        with open("temp.pdf", "wb") as f:
            f.write(pdf_response.content)
        text = ""
        with open("temp.pdf", "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print("Error extracting text from", pdf_url, ":", str(e))
        return None

# Iterate over each PDF URL in the DataFrame and extract text
for index, row in df.iterrows():
    pdf_url = row["pdf_url"]
    extracted_text = extract_text_from_pdf(pdf_url)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mhc.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mhc.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mhc.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: I

Error extracting text from scourtapp.nic.in/supremecourt/2021/32275/32275_2021_4_1509_43078_Judgement_24-Mar-2023.pdf : Invalid URL 'scourtapp.nic.in/supremecourt/2021/32275/32275_2021_4_1509_43078_Judgement_24-Mar-2023.pdf': No scheme supplied. Perhaps you meant https://scourtapp.nic.in/supremecourt/2021/32275/32275_2021_4_1509_43078_Judgement_24-Mar-2023.pdf?


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1

Error extracting text from https://main.sci.gov.in/supremecourt/2020/17613/17613_2020_3_301_43943_Order_24-Apr-2023.pdf  : EOF marker not found


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1

Error extracting text from https:https://main.sci.gov.in/supremecourt/2020/3654/3654_2020_10_1506_36042_Judgement_20-May-2022.pdf/main.sci.gov.in/supremecourt/2022/40099/40099_2022_1_1501_49105_Judgement_13-Dec-2023.pdf : Invalid URL 'https:https://main.sci.gov.in/supremecourt/2020/3654/3654_2020_10_1506_36042_Judgement_20-May-2022.pdf/main.sci.gov.in/supremecourt/2022/40099/40099_2022_1_1501_49105_Judgement_13-Dec-2023.pdf': No host supplied


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1

In [21]:
df['preprocessed_text'] = extracted_text

In [22]:
pip install nltk

In [23]:
# Download NLTK resources excluding omw-1.4
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [24]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [25]:
df['word_count']=df['preprocessed_text'].apply((lambda x:len(nltk.word_tokenize(x))))
df['sentence_count']=df['preprocessed_text'].apply((lambda x:len(nltk.sent_tokenize(x))))
df.sample(5)

,_id,Case_No,Judge,pdf_url,Petitioner,Outcome,Correct (Y/N),preprocessed_text,word_count,sentence_count
257,661394841911448714742d6a,Crl.A. No.-003051-003052 / 2023,HON'BLE MR. JUSTICE A.S. BOPANNA,https://main.sci.gov.in/supremecourt/2023/3027...,PANKAJ BANSAL,Plaintiff/Appellant wins,Yes,2023 INSC 1049\n \n Reportable \n \nIN THE SU...,8607,202
79,65df0283bf957c5aaed914fa,TCA.No.46 of 2015,THE HON'BLE MR.JUSTICE R.SUDHAKAR,https://mhc.tn.gov.in/judis/index.php/casestat...,The Commissioner of Income Tax,Respondent/Defendant wins,YES,2023 INSC 1049\n \n Reportable \n \nIN THE SU...,8607,202
192,661394841911448714742d29,C.A. No.-006805-006805 / 2022,HON'BLE THE CHIEF JUSTICE,https://main.sci.gov.in/supremecourt/2021/1588...,ORISSA ADMINISTRATIVE TRIBUNAL BAR ASSOCIATION,Plaintiff/Appellant wins,No (appeal dismissed),2023 INSC 1049\n \n Reportable \n \nIN THE SU...,8607,202
113,65df03f1bf957c5aaed9153e,W.P. No.3027 of 2015,THE HONOURABLE MR.JUSTICE S.VAIDYANATHAN,https://mhc.tn.gov.in/judis/index.php/casestat...,Sri Lakshmi Granites and Tiles,Plaintiff/Appellant wins,YES,2023 INSC 1049\n \n Reportable \n \nIN THE SU...,8607,202
119,65df0439bf957c5aaed9154a,WP.No.2581 of 2015,THE HON'BLE MR.JUSTICE SATISH K. AGNIHOTRI,https://mhc.tn.gov.in/judis/index.php/casestat...,R.Balasubramaniam,Respondent/Defendant wins,YES,2023 INSC 1049\n \n Reportable \n \nIN THE SU...,8607,202


In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import wordnet

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)

    # Convert the text to lowercase
    tokens = [word.lower() for word in tokens]

    # Remove punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize the words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Apply preprocessing function to 'preprocessed_text' column
df['processed_text'] = df['preprocessed_text'].apply(preprocess_text)

In [27]:
# Check for NaN values in the entire DataFrame
nan_df = df.isna().any()

# Check for NaN values in each column
nan_columns = df.isna().any(axis=0)

# Check for NaN values in each row
nan_rows = df.isna().any(axis=1)

# Print the results
print("NaN values in the entire DataFrame:")
print(nan_df)

print("\nNaN values in each column:")
print(nan_columns)

print("\nNaN values in each row:")
print(nan_rows)

NaN values in the entire DataFrame:
_id                  False
Case_No              False
Judge                 True
pdf_url              False
Petitioner           False
Outcome              False
Correct (Y/N)        False
preprocessed_text    False
word_count           False
sentence_count       False
processed_text       False
dtype: bool

NaN values in each column:
_id                  False
Case_No              False
Judge                 True
pdf_url              False
Petitioner           False
Outcome              False
Correct (Y/N)        False
preprocessed_text    False
word_count           False
sentence_count       False
processed_text       False
dtype: bool

NaN values in each row:
0      False
1      False
2      False
3      False
4      False
       ...  
265    False
266    False
267    False
268    False
269    False
Length: 270, dtype: bool


In [28]:
df['Judge'].fillna('Unknown', inplace=True)


In [29]:
df['Outcome'] = df['Outcome'].replace("Plaintiff/Appellant Wins", "Plaintiff/Appellant wins")


In [30]:
df['Outcome'] = df['Outcome'].apply(lambda x: 1 if x == 'Plaintiff/Appellant wins' else (2 if x == 'Respondent/Defendant wins' else 0))


In [31]:
df['Outcome'].value_counts()

Outcome
1    208
2     61
0      1
Name: count, dtype: int64

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# Fit and transform the processed text data
tfidf_features = tfidf_vectorizer.fit_transform(df['processed_text'])

# Convert the TF-IDF matrix into a DataFrame
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with the original DataFrame
df = pd.concat([df, tfidf_df], axis=1)

In [33]:
# Drop columns for judge's name, petitioner, and case number
columns_to_drop = ['Judge', 'Petitioner', 'Case_No']
df = df.drop(columns=columns_to_drop)

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Fit and transform the preprocessed text data
X = tfidf_features

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Outcome'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [72]:
# Initialize the Logistic Regression model
lr_model = LogisticRegression()

# Train the model
lr_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = lr_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7592592592592593


In [36]:
import time
import platform
import sys
# Print environment details
print("Environment Details:")
print("- Python Version:", sys.version)
print("- Operating System:", platform.system())
print("- Processor:", platform.processor())

Environment Details:
- Python Version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
- Operating System: Linux
- Processor: x86_64


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

# Start time for training
train_start_time = time.time()

# Initialize the Logistic Regression model
lr_model = LogisticRegression()

# Train the model
lr_model.fit(X_train, y_train)

# End time for training
train_end_time = time.time()

# Start time for testing
test_start_time = time.time()

# Predict on the training set
y_train_pred = lr_model.predict(X_train)

# End time for testing
test_end_time = time.time()

# Calculate training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Predict on the testing set
y_test_pred = lr_model.predict(X_test)

# Calculate testing accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Testing Accuracy:", test_accuracy)

# Calculate execution time for training (in milliseconds)
train_execution_time = (train_end_time - train_start_time) * 1000
print("Training Execution Time (milliseconds): {:.2f}".format(train_execution_time))

# Calculate execution time for testing (in milliseconds)
test_execution_time = (test_end_time - test_start_time) * 1000
print("Testing Execution Time (milliseconds): {:.2f}".format(test_execution_time))


Training Accuracy: 0.7731481481481481
Testing Accuracy: 0.7592592592592593
Training Execution Time (milliseconds): 34.46
Testing Execution Time (milliseconds): 0.86


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
print("Start Time (milliseconds): {:.2f}".format(start_time * 1000))

# Assuming X_train, X_test, y_train, y_test are already defined

# Train Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Predict on training set
lr_predictions_train = lr_model.predict(X_train)

# Predict on testing set
lr_predictions_test = lr_model.predict(X_test)

# Calculate training accuracy
lr_accuracy_train = accuracy_score(y_train, lr_predictions_train)

# Calculate testing accuracy
lr_accuracy_test = accuracy_score(y_test, lr_predictions_test)

print("Logistic Regression Model Training Accuracy:", lr_accuracy_train)
print("Logistic Regression Model Testing Accuracy:", lr_accuracy_test)

end_time = time.time()
print("End Time (milliseconds): {:.2f}".format(end_time * 1000))
execution_time_ms = (end_time - start_time) * 1000

print("Execution Time (milliseconds):", execution_time_ms)


Start Time (milliseconds): 1713801002051.30
Logistic Regression Model Training Accuracy: 0.7731481481481481
Logistic Regression Model Testing Accuracy: 0.7592592592592593
End Time (milliseconds): 1713801002085.13
Execution Time (milliseconds): 33.829450607299805


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

# Loop for running the code 10 times
for i in range(10):
    print("Iteration:", i+1)
    start_time = time.time()
    print("Start Time (milliseconds): {:.2f}".format(start_time * 1000))

    # Assuming X_train, X_test, y_train, y_test are already defined

    # Train Logistic Regression model
    lr_model = LogisticRegression()
    lr_model.fit(X_train, y_train)

    # Predict on training set
    lr_predictions_train = lr_model.predict(X_train)

    # Predict on testing set
    lr_predictions_test = lr_model.predict(X_test)

    # Calculate training accuracy
    lr_accuracy_train = accuracy_score(y_train, lr_predictions_train)

    # Calculate testing accuracy
    lr_accuracy_test = accuracy_score(y_test, lr_predictions_test)

    print("Logistic Regression Model Training Accuracy:", lr_accuracy_train)
    print("Logistic Regression Model Testing Accuracy:", lr_accuracy_test)

    end_time = time.time()
    exe = (end_time - start_time) * 1000
    print("Execution Time (milliseconds): {:.2f}".format(exe))
    print("End Time (milliseconds): {:.2f}".format(end_time * 1000))
    print()

Iteration: 1
Start Time (milliseconds): 1713801002108.68
Logistic Regression Model Training Accuracy: 0.7731481481481481
Logistic Regression Model Testing Accuracy: 0.7592592592592593
Execution Time (milliseconds): 36.79
End Time (milliseconds): 1713801002145.47

Iteration: 2
Start Time (milliseconds): 1713801002145.56
Logistic Regression Model Training Accuracy: 0.7731481481481481
Logistic Regression Model Testing Accuracy: 0.7592592592592593
Execution Time (milliseconds): 35.13
End Time (milliseconds): 1713801002180.69

Iteration: 3
Start Time (milliseconds): 1713801002180.79
Logistic Regression Model Training Accuracy: 0.7731481481481481
Logistic Regression Model Testing Accuracy: 0.7592592592592593
Execution Time (milliseconds): 38.90
End Time (milliseconds): 1713801002219.69

Iteration: 4
Start Time (milliseconds): 1713801002219.79
Logistic Regression Model Training Accuracy: 0.7731481481481481
Logistic Regression Model Testing Accuracy: 0.7592592592592593
Execution Time (millisec

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time
import pandas as pd

start_time = time.time()
print("Start Time (milliseconds): {:.2f}".format(start_time * 1000))

# Assuming X_train, X_test, y_train, y_test are already defined

# Train Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Predict on training set
lr_predictions_train = lr_model.predict(X_train)

# Predict on testing set
lr_predictions_test = lr_model.predict(X_test)

# Calculate training accuracy
lr_accuracy_train = accuracy_score(y_train, lr_predictions_train)

# Calculate testing accuracy
lr_accuracy_test = accuracy_score(y_test, lr_predictions_test)

print("Logistic Regression Model Training Accuracy:", lr_accuracy_train)
print("Logistic Regression Model Testing Accuracy:", lr_accuracy_test)

# Create DataFrame to store actual and predicted outcomes
results_df = pd.DataFrame({'Output Label': y_test, 'Predicted Outcome': lr_predictions_test})

# Print the DataFrame
print("Actual and Predicted Outcomes:")
print(results_df)

end_time = time.time()
print("End Time (milliseconds): {:.2f}".format(end_time * 1000))
execution_time_ms = (end_time - start_time) * 1000

print("Execution Time (milliseconds):", execution_time_ms)


Start Time (milliseconds): 1713801003006.01
Logistic Regression Model Training Accuracy: 0.7731481481481481
Logistic Regression Model Testing Accuracy: 0.7592592592592593
Actual and Predicted Outcomes:
    Output Label  Predicted Outcome
0              1                  1
1              1                  1
2              2                  1
3              1                  1
4              1                  1
5              1                  1
6              1                  1
7              1                  1
8              1                  1
9              1                  1
10             1                  1
11             1                  1
12             1                  1
13             1                  1
14             2                  1
15             1                  1
16             1                  1
17             1                  1
18             1                  1
19             2                  1
20             1                  1
21    

In [43]:
test_df=pd.read_excel("101. Hackathon Testing Dataset.xlsx")

In [44]:
test_df.shape

(100, 9)

In [45]:
test_df.columns

Index(['Sr. No.', 'Court', 'Case Type', 'Date', 'Case Number', 'Appellant',
       'Respondent', 'Outcome', 'Link to PDF'],
      dtype='object')

In [46]:
import zipfile

In [47]:
zip_ref = zipfile.ZipFile("/content/Lex Res-Amrita Hackathon-20240421T154843Z-001 (1).zip", 'r')
zip_ref.extractall("/content/Lex Res-Amrita Hackathon-20240421T154843Z-001 (1)")
zip_ref.close()

In [48]:
test_path="/content/Lex Res-Amrita Hackathon-20240421T154843Z-001/Lex Res-Amrita Hackathon"

In [49]:
pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 927.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 102.8 MB/s eta 0:00:00


In [50]:
import os
import pdfplumber

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Function to preprocess all PDF files in a directory
def preprocess_pdfs_in_directory(directory):
    pdf_texts = []
    # Get a sorted list of filenames
    filenames = sorted(os.listdir(directory))
    for filename in filenames:
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            text = extract_text_from_pdf(pdf_path)
            pdf_texts.append(text)
    return pdf_texts

# Path to the directory containing PDF files
test_path = "/content/Lex Res-Amrita Hackathon-20240421T154843Z-001 (1)/Lex Res-Amrita Hackathon"

# Preprocess PDFs in the directory
pdf_texts = preprocess_pdfs_in_directory(test_path)


In [51]:
pdf_texts

['http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 1 of 3\nPETITIONER:\nASHOK PAPER MILLS KAMGAR UNION\nVs.\nRESPONDENT:\nUNION OF INDIA & ANR.\nDATE OF JUDGMENT: 01/09/2000\nBENCH:\nS. Rajendra Babu, J. & S.N. Phukan, J.\nJUDGMENT:\nRAJENDRA BABU, J. :\nL...I...T.......T.......T.......T.......T.......T.......T..J\nBy an order made on July 6, 1996, this Court finalised\nthe scheme of rehabilitation of Ashok Paper Mills in\nDarbhanga in State of Bihar. However, that scheme could not\nbe given effect to due to one reason or the other and this\nCourt on April 26, 1999 considered various aspects of the\nmatter and made an order on I.A.No.13 in W.P. (C) No.\n174/91.\nAfter considering the various circumstances, this Court\ndirected that State of Bihar should stand guarantee for the\nsupply of four diesel generating sets to the tune of three\ncrores so that the difficulty in getting the power supply\nwould stand obviated and the company should give the\nproposal to the State of Bihar in that 

In [52]:
test_df['extracted_text'] = pdf_texts

In [53]:
test_df.columns

Index(['Sr. No.', 'Court', 'Case Type', 'Date', 'Case Number', 'Appellant',
       'Respondent', 'Outcome', 'Link to PDF', 'extracted_text'],
      dtype='object')

In [54]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import wordnet

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)

    # Convert the text to lowercase
    tokens = [word.lower() for word in tokens]

    # Remove punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize the words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Apply preprocessing function to 'preprocessed_text' column
test_df['extracted_text'] = test_df['extracted_text'].apply(preprocess_text)

In [55]:
test_df['extracted_text'].head(1)

0    http supreme court india page petitioner ashok...
Name: extracted_text, dtype: object

In [56]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming df is your DataFrame containing 'processed_text' column

# Initialize TF-IDF Vectorizer
test_tfidf_vectorizer = TfidfVectorizer(max_features=761)
# Fit and transform the processed text data
test_tfidf_features = test_tfidf_vectorizer.fit_transform(test_df['extracted_text'])

# Convert the TF-IDF matrix into a DataFrame
test_tfidf_df = pd.DataFrame(test_tfidf_features.toarray(), columns=test_tfidf_vectorizer.get_feature_names_out())

# Reset the index of the original DataFrame
test_df.reset_index(drop=True, inplace=True)

# Reset the index of tfidf_df
test_tfidf_df.reset_index(drop=True, inplace=True)

# Concatenate the TF-IDF features with the original DataFrame
test_df = pd.concat([test_df, test_tfidf_df], axis=1)


In [57]:
test_df['Outcome'].value_counts()

Outcome
Respondent Wins                                                               32
petitioner                                                                    16
Appellant Wins                                                                14
Appellant wins                                                                 5
respondent                                                                     4
Respondent wins                                                                3
Petitioner won                                                                 2
Appelant wins                                                                  2
Petitioner                                                                     2
Respondent Wins, Dismissed                                                     2
Appellant                                                                      1
Respondent Wins - Dismissed                                                    1
Respondent Wins - Di

In [58]:
# Define a mapping for similar outcomes to the three classes
outcome_mapping = {
    "Respondent Wins": "Respondent/Defendant wins",
    "petitioner": "Plaintiff/Appellant wins",
    "Appellant Wins": "Plaintiff/Appellant wins",
    "Appellant wins": "Plaintiff/Appellant wins",
    "respondent": "Respondent/Defendant wins",
    "Respondent wins": "Respondent/Defendant wins",
    "Petitioner won": "Plaintiff/Appellant wins",
    "Appelant wins": "Plaintiff/Appellant wins",
    "Petitioner": "Plaintiff/Appellant wins",
    "Respondent Wins, Dismissed": "Respondent/Defendant wins",
    "Appellant": "Plaintiff/Appellant wins",
    "Respondent Wins - Dismissed": "Respondent/Defendant wins",
    "Respondent Wins - Dismissed with directions to Respondents": "Respondent/Defendant wins",
    "Respondent Wins - Directions to Appellant": "Respondent/Defendant wins",
    "Appellant Wins - Regular Civil Appeal restored to District Judge": "Plaintiff/Appellant wins",
    "Respondent won": "Respondent/Defendant wins",
    "Appellant Wins - Direction to NCLT": "Plaintiff/Appellant wins",
    "Appellant wins. Appeal allowed in part": "Plaintiff/Appellant wins",
    "Appellant Wins, Allowed in part": "Plaintiff/Appellant wins",
    "Appellant wins": "Plaintiff/Appellant wins",
    "Respondent Wins, Dismissed - No merit": "Respondent/Defendant wins",
    "Appellant Wins, Appeal partly allowed": "Plaintiff/Appellant wins",
    "Appellant Wins, Partly Suceeds": "Plaintiff/Appellant wins",
    "Respondents Win": "Respondent/Defendant wins",
    "Appellant Wins, allowed in part": "Plaintiff/Appellant wins",
    "Appellant Wins, With direction to respondents": "Plaintiff/Appellant wins",
    "Both parties directed to file written statements. No judgement pronounced.": "Undetermined"
}

# Map similar outcomes to the three classes and set the rest to "Undetermined"
test_df['Outcome'] = test_df['Outcome'].map(outcome_mapping).fillna("Undetermined")

# Add an additional class "Undetermined" for outcomes not present in the mapping
test_df.loc[~test_df['Outcome'].isin(["Plaintiff/Appellant wins", "Respondent/Defendant wins", "Undetermined"]), 'Outcome'] = "Undetermined"


In [59]:
test_df['Outcome'].value_counts()

Outcome
Plaintiff/Appellant wins     50
Respondent/Defendant wins    47
Undetermined                  3
Name: count, dtype: int64

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Fit and transform the preprocessed text data
new_X = test_tfidf_features

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
new_y = label_encoder.fit_transform(test_df['Outcome'])



In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
print("Start Time (milliseconds): {:.2f}".format(start_time * 1000))

# Assuming X_train, X_test, y_train, y_test are already defined

# Train Logistic Regression model
lr_mode = LogisticRegression()
lr_model.fit(X_train, y_train)


# Predict on testing set
lr_predictions_test_new = lr_model.predict(new_X)

# Calculate testing accuracy
lr_accuracy_test_new = accuracy_score(new_y, lr_predictions_test_new)

print("Logistic Regression Model Testing Accuracy:", lr_accuracy_test_new)

end_time = time.time()
print("End Time (milliseconds): {:.2f}".format(end_time * 1000))


Start Time (milliseconds): 1713804984639.60
Logistic Regression Model Testing Accuracy: 0.47
End Time (milliseconds): 1713804984691.41


In [94]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

# Initialize variables to store cumulative accuracy and computation time
total_accuracy = 0
total_computation_time = 0

# Loop for running the code 10 times
for i in range(10):
    print("Iteration:", i+1)
    start_time = time.time()
    print("Start Time (milliseconds): {:.2f}".format(start_time * 1000))

    # Assuming X_train, X_test, y_train, y_test are already defined

   # Train Logistic Regression model
    lr_model = LogisticRegression()
    lr_model.fit(X_train, y_train)


# Predict on testing set
    lr_predictions_test_new = lr_model.predict(new_X)

# Calculate testing accuracy
    lr_accuracy_test_new = accuracy_score(new_y, lr_predictions_test_new)
    end_time = time.time()
    exe = (end_time - start_time) * 1000
    total_computation_time += exe

    print("Logistic Regression Model Testing Accuracy:", lr_accuracy_test_new)
    print("Execution Time (milliseconds): {:.2f}".format(exe))
    print("End Time (milliseconds): {:.2f}".format(end_time * 1000))
    print()

    # Accumulate accuracy for normalization
    total_accuracy += lr_accuracy_test



Iteration: 1
Start Time (milliseconds): 1713805055460.98
Logistic Regression Model Testing Accuracy: 0.47
Execution Time (milliseconds): 33.48
End Time (milliseconds): 1713805055494.46

Iteration: 2
Start Time (milliseconds): 1713805055494.63
Logistic Regression Model Testing Accuracy: 0.47
Execution Time (milliseconds): 30.25
End Time (milliseconds): 1713805055524.88

Iteration: 3
Start Time (milliseconds): 1713805055527.27
Logistic Regression Model Testing Accuracy: 0.47
Execution Time (milliseconds): 31.15
End Time (milliseconds): 1713805055558.42

Iteration: 4
Start Time (milliseconds): 1713805055559.42
Logistic Regression Model Testing Accuracy: 0.47
Execution Time (milliseconds): 30.74
End Time (milliseconds): 1713805055590.16

Iteration: 5
Start Time (milliseconds): 1713805055591.80
Logistic Regression Model Testing Accuracy: 0.47
Execution Time (milliseconds): 30.92
End Time (milliseconds): 1713805055622.71

Iteration: 6
Start Time (milliseconds): 1713805055623.71
Logistic Regr

In [95]:
# Create DataFrame to store actual and predicted outcomes
results_df = pd.DataFrame({'Output Label': new_y, 'Predicted Outcome': lr_predictions_test_new})

# Print the DataFrame
print("Actual and Predicted Outcomes:")
print(results_df)




Actual and Predicted Outcomes:
    Output Label  Predicted Outcome
0              1                  1
1              1                  1
2              1                  1
3              0                  1
4              0                  1
..           ...                ...
95             0                  1
96             1                  1
97             2                  1
98             0                  1
99             0                  1

[100 rows x 2 columns]


In [97]:
results_df.to_excel("output_labels.xlsx")